# Demo 1: phase conservation of backwards propagation

A demo to see if backwards propagation correctly calculates the initial phase of a point source.                     We generate three point sources, at phases of -3, 0 and 3 radians.
These fields are propagated to a plane above the point sources and propagated backwards.                             Then, the mean phase of the field below is calculated and compared to the original phase value.                     Finally, the phase differences are plotted as function of original phase.                                

In [ ]:
%matplotlib notebook

import matplotlib.pyplot as pt

import numpy as np
from pathlib import Path

from PyPO.System import System


s = System()

D = 100 # Distance between point source and upper screen.
    
source = {
        "name"      : "source",
        "gmode"     : "xy",
        "lims_x"    : np.array([-0.01, 0.01]),
        "lims_y"    : np.array([-0.01, 0.01]),
        "gridsize"  : np.array([31, 31])
        }

plane_up = {
        "name"      : "plane_up",
        "gmode"     : "uv",
        "lims_u"    : np.array([0, 100]),
        "lims_v"    : np.array([0, 360]),
        "gridsize"  : np.array([101, 100]),
        "flip"      : True
        }

plane_down = {
        "name"      : "plane_down",
        "gmode"     : "uv",
        "lims_u"    : np.array([0, 0.1]),
        "lims_v"    : np.array([0, 360]),
        "gridsize"  : np.array([101, 100])
        }

s.addPlane(source)
s.addPlane(plane_up)
s.addPlane(plane_down)

ph_diff = []
phases = np.linspace(-3, 3, 3)
for ph in phases:
    PSDict = {
            "name"      : "PS_source",
            "lam"       : 1,
            "E0"        : 1,
            "phase"     : ph,
            "pol"       : np.array([1,0,0])
            }

    s.createPointSource(PSDict, "source")

    s.translateGrids("plane_up", np.array([0, 0, 100]))

    runPODict = {
            "t_name"    : "plane_up",
            "s_current" : "PS_source",
            "epsilon"   : 10,
            "exp"       : "fwd",
            "mode"      : "JMEH",
            "name_JM"   : "JM_up",
            "name_EH"   : "EH_up",
            }

    runPODict_bwd = {
            "t_name"    : "plane_down",
            "s_current" : "JM_up",
            "epsilon"   : 10,
            "exp"       : "bwd",
            "mode"      : "JMEH",
            "name_JM"   : "JM_down",
            "name_EH"   : "EH_down",
            }

    s.runPO(runPODict)
    s.runPO(runPODict_bwd)

    # The mean phase is simply calculated as the means of the phase over the surface.
    phase_Ex = np.mean(np.angle(s.fields["EH_down"].Ex))
    ph_diff.append(ph - phase_Ex)

fig, ax = pt.subplots(1,1, figsize=(5,5))
ax.scatter(phases, ph_diff)
ax.set_xlabel(r"$\phi_0$ / radians")
ax.set_ylabel(r"$\Delta \phi$ / radians")
pt.show()

2025-07-09 15:36:39 - WARNING - System override set to True. 
2025-07-09 15:36:39 - INFO - Added plane source to system. 
2025-07-09 15:36:39 - INFO - Added plane plane_up to system. 
2025-07-09 15:36:39 - INFO - Added plane plane_down to system. 


FileNotFoundError: Could not find module 'C:\Users\Paul Grimes\miniforge\envs\PyPO\Lib\site-packages\PyPO\libpypobeam.dylib' (or one of its dependencies). Try using the full path with constructor syntax.

In the image above, $\phi_0$ represents the initial phase and $\Delta \phi = \phi_0 - \phi_\mathrm{bw}$ the phase difference between the initial phase and mean backwards propagated phase. It can be seen that the difference is small for all $\phi_0$.